In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import DCdetector as model
# from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_data = np.load('/home/iot/collision_detect/OmniAnomaly/processed/MSL_train.pkl',allow_pickle=True)
test_data  = np.load('/home/iot/collision_detect/OmniAnomaly/processed/MSL_test.pkl',allow_pickle=True)
test_label = np.load('/home/iot/collision_detect/OmniAnomaly/processed/MSL_test_label.pkl',allow_pickle=True)

In [3]:
clf = model(epochs=5,seq_len=100,stride=10)
clf.fit(train_data)

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5


In [6]:
scores        = clf.decision_function(test_data)
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(test_label, scores)
result_audio = eval_metrics_audio
print(result_audio)

(0.4949852202357459, 0.10425564489039352, 0.19088881726802953, 0.10551630434782609, 1.0)


In [7]:
adj_eval_metrics = ts_metrics(test_label, point_adjustment(test_label, scores))
print(adj_eval_metrics)

(0.9829370597566405, 0.862739198382023, 0.9006866093128802, 0.941069173565315, 0.8636363636363636)


In [6]:
def get_sub_seqs_gt(x_arr, seq_len=100, stride=1):
    """

    Parameters
    ----------
    x_arr: np.array, required
        input original data with shape [time_length, channels]

    seq_len: int, optional (default=100)
        Size of window used to create subsequences from the data
get_sub_seqs_label
    stride: int, optional (default=1)
        number of time points the window will move between two subsequences

    Returns
    -------
    x_seqs: np.array
        Split sub-sequences of input time-series data
    """
    print(x_arr.shape[0])
    if x_arr.shape[0] < seq_len:
        seq_len = x_arr.shape[0]
    seq_starts = np.arange(0, x_arr.shape[0] - seq_len + 1, stride)
    x_seqs = np.array([x_arr[i:i + seq_len] for i in seq_starts])

    return x_seqs


def get_sub_seqs_label(y, seq_len=100, stride=1):
    """

    Parameters
    ----------
    y: np.array, required
        data labels

    seq_len: int, optional (default=100)
        Size of window used to create subsequences from the data

    stride: int, optional (default=1)
        number of time points the window will move between two subsequences

    Returns
    -------
    y_seqs: np.array
        Split label of each sequence
    """
    if y.shape[0] < seq_len:
        seq_len = y.shape[0]

    seq_starts = np.arange(0, y.shape[0] - seq_len + 1, stride)
    ys = np.array([y[i:i + seq_len] for i in seq_starts])
    y = np.sum(ys, axis=1) / seq_len

    y_binary = np.zeros_like(y)
    y_binary[np.where(y!=0)[0]] = 1
    return y_binary

In [7]:
test_label_n = get_sub_seqs_gt(test_label,100,10)

73729


In [8]:
test_label_n = np.sum(test_label_n,axis=-1)

In [11]:
test_label_n[test_label_n!=0]=1

In [12]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(test_label_n, scores)
result_audio = eval_metrics_audio
print(result_audio)

(0.5015143879625311, 0.15475189278318668, 0.2663033190589378, 0.15360586717370636, 1.0)


In [ ]:
adj_eval_metrics = ts_metrics(test_label, point_adjustment(test_label_n, scores))
print(adj_eval_metrics)